In [1]:
import os
import sys
import struct
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
# MNISTのファイル (あらかじめダウンロードしておく)
train_image_file = 'mnist/train-images-idx3-ubyte'
train_label_file = 'mnist/train-labels-idx1-ubyte'
test_image_file = 'mnist/t10k-images-idx3-ubyte'
test_label_file = 'mnist/t10k-labels-idx1-ubyte'

In [3]:
def load_images(filename):
    """ MNISTの画像データを読み込む """

    fp = open(filename, 'rb')
    
    # マジックナンバー
    magic = struct.unpack('>i', fp.read(4))[0]
    if magic != 2051:
        raise Exception('Invalid MNIST file!')
        
    # 各種サイズ
    n_images, height, width = struct.unpack('>iii', fp.read(4 * 3))
    
    # 画像の読み込み
    total_pixels = n_images * height * width
    images = struct.unpack('>' + 'B' * total_pixels, fp.read(total_pixels))
    
    images = np.asarray(images, dtype='uint8')
    images = images.reshape((n_images, height, width, 1))
    
    # 値の範囲を[0, 1]に変更する
    images = images.astype('float32') / 255.0
    
    fp.close()
    
    return images

In [4]:
def load_labels(filename):
    """ MNISTのラベルデータを読み込む """

    fp = open(filename, 'rb')
    
    # マジックナンバー
    magic = struct.unpack('>i', fp.read(4))[0]
    if magic != 2049:
        raise Exception('Invalid MNIST file!')
        
    # 各種サイズ
    n_labels = struct.unpack('>i', fp.read(4))[0]
    
    # ラベルの読み込み
    labels = struct.unpack('>' + 'B' * n_labels, fp.read(n_labels))
    labels = np.asarray(labels, dtype='int32')
    
    fp.close()
    
    return labels

In [5]:
def to_onehot(labels):
    """ one-hot形式への変換 """
    return np.identity(10)[labels]

## モデルの訓練

In [6]:
images = load_images(train_image_file)
labels = load_labels(train_label_file)
onehot = to_onehot(labels)

In [7]:
num_data = len(images)
X = images.reshape((num_data, -1))
y = onehot.reshape((num_data, -1))

In [8]:
def softmax(x, axis=-1):
    """ softmax関数 """
    x = x - np.max(x, axis=axis, keepdims=True)
    ex = np.exp(x)
    return ex / np.sum(ex, axis=axis, keepdims=True)

def logsumexp(x, axis=-1):
    """ logsumexp関数 """
    x_max = np.max(x, axis=axis, keepdims=True)
    x = x - x_max
    return x_max + np.log(np.sum(np.exp(x), axis=axis, keepdims=True))

def log_softmax(x, axis=-1):
    """ log-softmax関数 """
    ex = np.exp(x)
    return x - logsumexp(x, axis=axis)

In [9]:
# ミニバッチのサイズ
epochs = 6
batch_size = 32

# モメンタムSGDのパラメータ
alpha = 0.01
momentum = 0.9

# パラメータの初期化
in_features = X.shape[-1]
out_features = y.shape[-1]
AA = np.random.normal(size=(out_features, in_features)) / np.sqrt(0.5 * (out_features + in_features))
bb = np.zeros((out_features))

# エポック
grad_AA = np.zeros_like(AA)
grad_bb = np.zeros_like(bb)
for epoch in range(epochs):
    # データの順番は偏りをなくすためにランダムシャッフルする
    indices = np.random.permutation(np.arange(num_data))
    for b in range(0, num_data, batch_size):
        bs = min(batch_size, num_data - b)        
        X_batch = X[indices[b:b+bs], :]
        y_batch = y[indices[b:b+bs], :]

        loss = 0.0
        acc = 0.0
        grad_AA_new = np.zeros_like(AA)
        grad_bb_new = np.zeros_like(bb)
        
        # バッチ内の各データに対してロス、精度、勾配を求める
        t = np.einsum('ij,bj->bi', AA, X_batch) + bb
        log_y_pred = log_softmax(t)
        losses = np.sum(-y_batch * log_y_pred, axis=-1, keepdims=True)

        delta = np.identity(AA.shape[0])
        y_pred = np.exp(log_y_pred)
        dLdy = -y_batch / y_pred
        dydt = np.einsum('bi,ij->bij', y_pred, delta) - np.einsum('bi,bj->bij', y_pred, y_pred)
        dLdt = np.einsum('bi,bij->bj', dLdy, dydt)

        dtdA = np.einsum('bk,ij->bijk', X_batch, delta)
        dtdb = np.ones((bb.shape[-1], bb.shape[-1]))
        dLdA = np.einsum('bi,bijk->bjk', dLdt, dtdA)
        dLdb = np.einsum('bi,ij->bj', dLdt, dtdb)
            
        y_pred_id = np.argmax(y_pred, axis=-1)
        y_real_id = np.argmax(y_batch, axis=-1)
        acc = np.mean(y_pred_id == y_real_id)
        loss = np.mean(losses)

        grad_AA_new = np.mean(dLdA, axis=0)
        grad_bb_new = np.mean(dLdb, axis=0)
        
        # 勾配の更新
        grad_AA = grad_AA * momentum + grad_AA_new * (1.0 - momentum)
        grad_bb = grad_bb * momentum + grad_bb_new * (1.0 - momentum)

        # 最急降下法による値の更新
        AA -= alpha * grad_AA
        bb -= alpha * grad_bb
        
        # printの代わりにsys.stdout.writeを使うとcarrige returnが使える
        sys.stdout.write('\repoch:{}, step:{}, loss={:.6f}, acc={:.6f}'.format(epoch + 1, b + bs, loss, acc))
        sys.stdout.flush()

    sys.stdout.write('\n')
    sys.stdout.flush()

epoch:1, step:60000, loss=0.596077, acc=0.812500
epoch:2, step:60000, loss=0.399621, acc=0.906250
epoch:3, step:60000, loss=0.250540, acc=0.906250
epoch:4, step:60000, loss=0.251255, acc=0.968750
epoch:5, step:60000, loss=0.466148, acc=0.906250
epoch:6, step:60000, loss=0.312721, acc=0.937500


## モデルのテスト

In [10]:
test_images = load_images(test_image_file)
test_labels = load_labels(test_label_file)
test_onehot = to_onehot(test_labels)

In [11]:
num_data = len(images)
X = images.reshape((num_data, -1))
y = onehot.reshape((num_data, -1))

In [12]:
loss = 0.0
acc = 0.0
for b in range(0, num_data, batch_size):
    bs = min(batch_size, num_data - b)        
    X_batch = X[indices[b:b+bs], :]
    y_batch = y[indices[b:b+bs], :]
    
    t = np.einsum('ij,bj->bi', AA, X_batch) + bb
    log_y_pred = log_softmax(t)
    losses = np.sum(-y_batch * log_y_pred, axis=-1, keepdims=True)

    y_pred = np.exp(log_y_pred)
    y_pred_id = np.argmax(y_pred, axis=-1)
    y_real_id = np.argmax(y_batch, axis=-1)

    acc += np.sum(y_pred_id == y_real_id)
    loss += np.sum(losses)

loss /= num_data
acc /= num_data

In [13]:
print('Loss: {:.4f}'.format(loss))
print('Accuracy: {:.4f}'.format(acc))

Loss: 0.3465
Accuracy: 0.9039
